<a href="https://colab.research.google.com/github/pnjha/NLP-Assignments/blob/master/nlp12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive


In [2]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import copy
import math
import tensorflow as tf
from keras.optimizers import SGD
from keras.initializers import RandomUniform
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import RepeatVector, Bidirectional, Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout, Activation
from keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

Using TensorFlow backend.


In [3]:
X = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.en', names=['src'])
Y = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.hi', names=['dest'])
lines = pd.concat([X[:3000],Y[:3000]], axis=1)
print(len(lines))
# lines = shuffle(lines)
lines.sample(10)

3000


,src,dest
1290,There are three main kinds of this type of her...,इस तरह के हर्निया के भी मुख्यत : तीन प्रकार है...
1429,This disease occurs in extremely obese teenage...,यह बीमारी ऐसे हद से ज्यादा मोटे किशोरों को होत...
1341,Generally during hernioplasty there is a dange...,आमतौर पर हर्नियोप्लास्टी के दौरान सर्जरी वाले ...
2252,Urine may come out on its own when pressure is...,"मूत्राशय , मूत्र तंत्र पर जोर , तनाव पड़ने जैसे..."
2799,In the prostrate gland milk like white fluid s...,प्रोस्टेट ग्रंथि में दूध जैसा सफेद तरल पदार्थ ...
1919,Till that time we will not even be able to cou...,तब तक तो हम पाँच तक गिनती भी नहीं कर पाएँगे ।
1869,"Touch , heat , pressure , pricking everything ...","स्पर्श , ताप , दबाव , चुभन सब यहीं अनुभव होता ..."
2867,"Dirt , dust , smell of house , smoke of fuel c...","घर की गंदगी , धूल , सुगंध , ईंधन का धुआँ बच्चो..."
777,"Our eyes do not get spoiled by TV , computer o...","टीवी , कंम्प्यूटर या कम रोशनी से हमारी आँखें ख..."
70,"Generally , easy tumors can be weeded out thro...",सामान्यतः सुसाध्य ट्यूमरों को शल्य-चिकित्सा द्...


In [0]:
def process_data(data,append_char):
  data = data.apply(lambda x: x.lower())
  data = data.apply(lambda x: x.strip())
  data = data.apply(lambda x: re.sub("'", '', x))
  exclude = set(string.punctuation) # Set of all special characters
  data = data.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
  remove_digits = str.maketrans('', '', digits)
  data = data.apply(lambda x: x.translate(remove_digits))
  if append_char:
    data = data.apply(lambda x : '<SOS> '+ x + ' <EOS>')
  return data

In [5]:
lines.src = process_data(lines.src,False)
lines.dest = process_data(lines.dest,True)
lines.sample(10)

,src,dest
1464,and this is applied on the balding region,<SOS> और गंजेपन की जगह लगाते हैं । <EOS>
1199,the plant of mustard is green and can be fee...,<SOS> सरसों का पौधा से फीट तक लंबा और हरा हो...
1408,skip for minutes,<SOS> मिनट रस्सी कूदें । <EOS>
2954,all daily jobs done by humans are effected by ...,<SOS> इन्सान द्वारा किए गए सारे दैनिक कार्य शर...
2636,the pulse polio program was started on an expe...,<SOS> पल्स पोलियो कार्यक्रम सबसे पहले प्रायोग...
796,total hip knee replacement lrb tkr rrb is usef...,<SOS> समूचे जोड़ को प्रभावित करने वाले आर्थराइट...
2345,cancer of any organ in body spreads either thr...,<SOS> शरीर के किसी भी अंग का कैंसर या तो रक्‍त...
1541,watery dysentery starts coming,<SOS> पतले दस्त लग जाते हैं । <EOS>
2195,to take out the cataract with the help of the ...,<SOS> ’फैको इसल्सिफिकेशन’ तकनीक से मोतियाबिंद ...
196,chicken pox is more serious in adolescents and...,<SOS> बच्चों की तुलना मे छोटीमाता किशोरों और व...


In [0]:
def get_vocab_list(data):
  vocab = []
  for line in data:
    temp = line.split()
    for item in temp:
      if item not in vocab:
        vocab.append(item)
  return vocab

In [0]:
src_vocab = get_vocab_list(lines.src)
dest_vocab = get_vocab_list(lines.dest)

In [8]:
print(src_vocab[:10])
print(dest_vocab[:10]) 

['the', 'treatment', 'of', 'cataract', 'is', 'possible', 'through', 'surgery', 'only', 'complete']
['<SOS>', 'मोतियाबिंद', 'का', 'उपचार', 'केवल', 'शल्यचिकित्सा', 'द्वारा', 'ही', 'सम्भव', 'है']


In [0]:
# Max Length of source sequence
def get_max_sentence_size(data):
  max_len = 0
  for line in data:
    temp = line.split()
    max_len = max(max_len, len(temp))
  return max_len

In [10]:
src_max_len = get_max_sentence_size(lines.src)
dest_max_len = get_max_sentence_size(lines.dest)
src_max_len,dest_max_len

(84, 86)

In [11]:
src_vocab_len = len(src_vocab)
dest_vocab_len = len(dest_vocab) + 1
src_vocab_len, dest_vocab_len

(4711, 6289)

In [0]:
def get_word_to_index_dict(vocab):
  word_to_index = {}
  for i, word in enumerate(vocab):
    word_to_index[word] = i+1
  return word_to_index

In [0]:
src_word_to_index = get_word_to_index_dict(src_vocab)
dest_word_to_index = get_word_to_index_dict(dest_vocab)

In [14]:
print(src_word_to_index)
print(dest_word_to_index)

{'the': 1, 'treatment': 2, 'of': 3, 'cataract': 4, 'is': 5, 'possible': 6, 'through': 7, 'surgery': 8, 'only': 9, 'complete': 10, 'lens': 11, 'capsule': 12, 'taken': 13, 'out': 14, 'in': 15, 'method': 16, 'intra': 17, 'capsular': 18, 'extraction': 19, 'after': 20, 'operation': 21, 'spectacles': 22, 'are': 23, 'used': 24, 'posttest': 25, 'during': 26, 'implanted': 27, 'at': 28, 'front': 29, 'iris': 30, 'frontal': 31, 'part': 32, 'eye': 33, 'extra': 34, 'and': 35, 'whole': 36, 'matter': 37, 'extracted': 38, 'posterior': 39, 'left': 40, 'intact': 41, 'fitted': 42, 'bag': 43, 's': 44, 'lrb': 45, 'small': 46, 'incision': 47, 'rrb': 48, 'mm': 49, 'tunnel': 50, 'formed': 51, 'while': 52, 'rest': 53, 'follows': 54, 'like': 55, 'that': 56, 'ecceiol': 57, 'no': 58, 'stitches': 59, 'applied': 60, 'black': 61, 'nerves': 62, 'dilapidates': 63, 'gradually': 64, 'it': 65, 'happens': 66, 'due': 67, 'to': 68, 'water': 69, 'pressure': 70, 'or': 71, 'because': 72, 'not': 73, 'enough': 74, 'blood': 75, 'r

In [0]:
def get_index_to_word_dict(vocab):
  index_to_word = {}
  for i, word in enumerate(vocab):
    index_to_word[i+1] = word
  return index_to_word  

In [0]:
src_index_to_word = get_index_to_word_dict(src_vocab)
dest_index_to_word = get_index_to_word_dict(dest_vocab)

In [17]:
print(src_index_to_word)
print(dest_index_to_word)

{1: 'the', 2: 'treatment', 3: 'of', 4: 'cataract', 5: 'is', 6: 'possible', 7: 'through', 8: 'surgery', 9: 'only', 10: 'complete', 11: 'lens', 12: 'capsule', 13: 'taken', 14: 'out', 15: 'in', 16: 'method', 17: 'intra', 18: 'capsular', 19: 'extraction', 20: 'after', 21: 'operation', 22: 'spectacles', 23: 'are', 24: 'used', 25: 'posttest', 26: 'during', 27: 'implanted', 28: 'at', 29: 'front', 30: 'iris', 31: 'frontal', 32: 'part', 33: 'eye', 34: 'extra', 35: 'and', 36: 'whole', 37: 'matter', 38: 'extracted', 39: 'posterior', 40: 'left', 41: 'intact', 42: 'fitted', 43: 'bag', 44: 's', 45: 'lrb', 46: 'small', 47: 'incision', 48: 'rrb', 49: 'mm', 50: 'tunnel', 51: 'formed', 52: 'while', 53: 'rest', 54: 'follows', 55: 'like', 56: 'that', 57: 'ecceiol', 58: 'no', 59: 'stitches', 60: 'applied', 61: 'black', 62: 'nerves', 63: 'dilapidates', 64: 'gradually', 65: 'it', 66: 'happens', 67: 'due', 68: 'to', 69: 'water', 70: 'pressure', 71: 'or', 72: 'because', 73: 'not', 74: 'enough', 75: 'blood', 76

In [0]:
def reverse_train_sentences(data):
  l = data.values.tolist()
  t = []
  for item in l:
    p = item.strip().split()
    p.reverse()
    k = []
    for i in p:
      i = i.strip()
      if len(i) > 0:
        k.append(i)
    p = k
    p = " ".join(p)
    p = p.strip()
    t.append(p)
  df = pd.DataFrame({'data':t})
  return df.data  

In [19]:
# Train - Test Split
X, y = lines.src, lines.dest
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1,shuffle=False)
X_train_normal = copy.deepcopy(X_train)
X_train = reverse_train_sentences(X_train)
print(X_train.shape, X_test.shape)
X_train.sample(10)

(2700,) (300,)


2647    bengal in found were and pradesh uttar found w...
276     emergence of frequency the on dependent are sy...
455     level national the at percent is this while ut...
496     women and men is both symptoms vagina the of d...
2396    way better much a in activity daily their fini...
2263    possible is problem this of treatment the but ...
2349    done cancer for test the get definitely and ca...
1668    fig eating by cured be can constipation perpet...
1763    direction opposite the in valley synaptic the ...
1716    satisfactory is itself in treatment this time ...
Name: data, dtype: object

In [0]:
def generate_batch(X = X_train, y = y_train):
  batch_size = len(X)
  encoder_input_data = np.zeros((batch_size, src_max_len),dtype='float32')
  decoder_target_data = np.zeros((batch_size, dest_max_len, dest_vocab_len),dtype='float32')
  decoder_data = np.zeros((batch_size, dest_max_len),dtype='float32')

  for i, (input_text, target_text) in enumerate(zip(X,y)):
      for t, word in enumerate(input_text.split()):
          encoder_input_data[i, t] = src_word_to_index[word]
      for t, word in enumerate(target_text.split()):
          decoder_target_data[i, t-1, dest_word_to_index[word]] = 1
          decoder_data[i,t] = dest_word_to_index[word]
  return encoder_input_data, decoder_target_data, decoder_data

In [0]:
latent_dim = 300

In [33]:
model = Sequential()

# Creating encoder network
model.add(Embedding(input_dim = src_vocab_len, output_dim = 300, input_length = src_max_len, mask_zero = True))
model.add(LSTM(units = latent_dim,return_sequences=True))
model.add(LSTM(units = latent_dim))

# Creating decoder network
model.add(RepeatVector(dest_max_len))
model.add(LSTM(units = latent_dim, return_sequences=True))
model.add(LSTM(units = latent_dim, return_sequences=True))
model.add(Dense(dest_vocab_len,activation='softmax'))
# model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer="rmsprop",metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 84, 300)           1413300   
_________________________________________________________________
lstm_5 (LSTM)                (None, 84, 300)           721200    
_________________________________________________________________
lstm_6 (LSTM)                (None, 300)               721200    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 86, 300)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 86, 300)           721200    
_________________________________________________________________
lstm_8 (LSTM)                (None, 86, 300)           721200    
_________________________________________________________________
dense_2 (Dense)              (None, 86, 6289)         

In [0]:
X,y, yp = generate_batch(X_train, y_train)
xtest, ytest, ytestp = generate_batch(X_test, y_test)

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
epochs = 30
batch_size = 128

In [36]:
model.fit(X,y,
          epochs=epochs,
          batch_size=batch_size,
          validation_data =(xtest,ytest))

Train on 2700 samples, validate on 300 samples
Epoch 1/30
2700/2700 [==============================] - 22s 8ms/step - loss: 1.5567 - acc: 0.0110 - val_loss: 1.4353 - val_acc: 0.0200
Epoch 2/30
2700/2700 [==============================] - 18s 7ms/step - loss: 1.4395 - acc: 0.0131 - val_loss: 1.4396 - val_acc: 0.0155
Epoch 3/30
2700/2700 [==============================] - 19s 7ms/step - loss: 1.4246 - acc: 0.0165 - val_loss: 1.4320 - val_acc: 0.0193
Epoch 4/30
2700/2700 [==============================] - 19s 7ms/step - loss: 1.4059 - acc: 0.0201 - val_loss: 1.4193 - val_acc: 0.0255
Epoch 5/30
2700/2700 [==============================] - 19s 7ms/step - loss: 1.3912 - acc: 0.0218 - val_loss: 1.4356 - val_acc: 0.0170
Epoch 6/30
2700/2700 [==============================] - 18s 7ms/step - loss: 1.3735 - acc: 0.0250 - val_loss: 1.4121 - val_acc: 0.0289
Epoch 7/30
2700/2700 [==============================] - 18s 7ms/step - loss: 1.3539 - acc: 0.0265 - val_loss: 1.4028 - val_acc: 0.0300
Epoch 8/

In [0]:
def calculate_bleu_score(actual_string, predicted_string):
  actual_string = copy.deepcopy(actual_string)
  predicted_string = copy.deepcopy(predicted_string)
  reference = re.split("\s",actual_string.strip())
  candidate = re.split("\s",predicted_string.strip())
  smoothie = SmoothingFunction().method4
  score = sentence_bleu(reference, candidate, smoothing_function=smoothie)
  return score

In [0]:
def decode_input_sentence(arr,inverse_mapping,skip=False):
  line = ""
  for i in range(len(arr)):
    if arr[i] == 0.0:
      break
    if i == 0 and skip == True:
      continue
    if i == len(arr)-1 and skip == True:
      continue  
    line += inverse_mapping[arr[i]]+" "
    
  return line

In [0]:
def decode_model_output(model_output):
  # print(model_output)
  line = ""
  for i in range(len(model_output)):
    max_index = np.argmax(model_output[i, :])
    word = dest_index_to_word[max_index+1]
    if word == "<EOS>":
      # print(i)
      break
    line += word+" "
  return line

In [32]:

yhat = model.predict(xtest, verbose=0)
bleu_score = 0.0
for i in range(len(yhat)):
  src_input_sentence = decode_input_sentence(xtest[i,:],src_index_to_word)
  #print(ytestp[i,:])
  dest_input_sentence = decode_input_sentence(ytestp[i,:],dest_index_to_word,True)
  output_sentence = decode_model_output(yhat[i,:])
  print("Actual Source: ",src_input_sentence)
  print("Actual Target: ",dest_input_sentence)
  print("predicted Target: ",output_sentence)
  curr_bs = calculate_bleu_score(dest_input_sentence,output_sentence)
  bleu_score += curr_bs
  print("Current BLEU Score: ",curr_bs)
  # break
print("Overall BLEU Score: ", bleu_score/len(yhat))

Actual Source:  high blood pressure is dangerous for health 
Actual Target:  उच्च रक्‍तचाप है स्वास्थ्य के लिए खतरा <EOS> 
predicted Target:  बाद बाद बाद पूरा । । 
Current BLEU Score:  0
Actual Source:  blood pressure is not such an illness that can not be controlled 
Actual Target:  रक्‍तचाप ऐसा रोग नहीं है जिसे नियंत्रित न किया जा सके । <EOS> 
predicted Target:  बाद बाद बाद बाद या या या । । । 
Current BLEU Score:  0.19552795980276136
Actual Source:  by adopting a course of abstinence and medicines it is possible to prevent the serious consequences of blood pressure 
Actual Target:  परहेज एवं दवाओं के सेवन से रक्‍तचाप के भयानक परिणामों से बचा जा सकता है । <EOS> 
predicted Target:  बाद बाद बाद बाद बाद या या या या या । । । । । 
Current BLEU Score:  0.18827362913939635
Actual Source:  blood pumped from left chamber of the heart circulates in the arteries spread throughout the body and it is the very pressure exerted on the walls of the arteries by blood circulation which is called blood 